In [9]:
import tvm
from tvm import te
import numpy as np

def conv_out_size(n,k,p,s):
    return (n-k+2*p)//s + 1
def padding(X, ph, pw, val=0):
    assert len(X.shape) >= 2
    nh,nw = X.shape[-2],X.shape[-1]
    return te.compute(
        (*X.shape[0:-2],nh+ph*2,nw+pw*2),
        lambda *i:te.if_then_else(
            te.any(i[-2]<ph, i[-2]>=nh+ph, i[-1]<pw, i[-1]>=nw+pw),
            val,X[i[:-2]+(i[-2]-ph,i[-1]-pw)]),name='PaddedX')

In [10]:
def pool(pool_type, c, nh, nw, kh, kw, ph=0, pw=0, sh=1, sw=1):
    # reduction axes
    rkh = te.reduce_axis((0,kh), name='rkh')
    rkw = te.reduce_axis((0,kw), name='rkw')
    
    oh = conv_out_size(nh, kh, ph, sh)
    ow = conv_out_size(nw, kw, pw, sw)
    
    X = te.placeholder((c, nh, nw), name='X')
    
    if pool_type == 'max':
        PaddedX = padding(X, ph, pw, val=te.min_value(X.dtype)) \
            if ph * pw != 0 else X
        Y = te.compute((c, oh, ow), \
                          lambda c, h, w:\
                          te.max(PaddedX[c, h*sh+rkh, w*sw+rkw],\
                                axis=[rkh,rkw]),\
                          tag='pool_max', name='PoolMax')
    elif pool_type == 'avg':
        PaddedX = padding(X, ph, pw) if ph * pw != 0 else X
        tsum = te.compute((c, oh, ow), \
                          lambda c, h, w:\
                          te.sum(PaddedX[c, h*sh+rkh, w*sw+rkw],\
                                axis=[rkh,rkw]),\
                          tag='pool_avg1', name='PoolSum')
        Y = te.compute((c,oh,ow),\
                          lambda c, h, w:\
                          tsum[c,h,w] / (kh*kw),\
                          tag='pool_avg2', name='PoolAvg')
    else:
        raise ValueError("Pool type should be 'avg' or 'max'.")
    return X, Y, PaddedX
        


In [11]:
def get_conv_data(oc,ic,n,k,p=0,s=1,constructor=None,conv_type='direct'):
    np.random.seed(0)
    data = np.random.normal(size=(ic,n,n)).astype('float32')
    ic_weight = ic
    if conv_type == 'depthwise':
        ic_weight = 1
    weight = np.random.normal(size=(oc,ic_weight,k,k)).astype('float32')
    on = conv_out_size(n,k,p,s)
    out = np.empty((oc,on,on), dtype='float32')
    if constructor:
        data, weight, out = (constructor(x) for x in [data, weight, out])
    return data,weight,out

In [12]:
c, n, k, p, s = 4, 12, 3, 1, 1
X, Y, PaddedX = pool('max', c, n, n, k, k, p, p, s, s)
sch = te.create_schedule(Y.op)
mod = tvm.build(sch, [X, Y])
print(tvm.lower(sch, [X, Y], simple_mode=True))
data, _, out_max = get_conv_data(c, c, n, k, p, s, tvm.nd.array)
mod(data, out_max)

@main = primfn(X_1: handle, PoolMax_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {X: Buffer(X_2: Pointer(float32), float32, [576], []),
             PoolMax: Buffer(PoolMax_2: Pointer(float32), float32, [576], [])}
  buffer_map = {X_1: X, PoolMax_1: PoolMax}
  preflattened_buffer_map = {X_1: X_3: Buffer(X_2, float32, [4, 12, 12], []), PoolMax_1: PoolMax_3: Buffer(PoolMax_2, float32, [4, 12, 12], [])} {
  allocate(PaddedX: Pointer(global float32), float32, [784]), storage_scope = global {
    for (i0: int32, 0, 4) {
      for (i1: int32, 0, 14) {
        for (i2: int32, 0, 14) {
          PaddedX_1: Buffer(PaddedX, float32, [784], [])[(((i0*196) + (i1*14)) + i2)] = @tir.if_then_else(((((i1 < 1) || (13 <= i1)) || (i2 < 1)) || (13 <= i2)), -3.40282e+38f32, X[((((i0*144) + (i1*12)) + i2) - 13)], dtype=float32)
        }
      }
    }
    for (c: int32, 0, 4) {
      for (h: int32, 0, 12) {
        for (w: int32, 0, 12